## Clean the Data

In [12]:
import sqlite3
from pydantic import BaseModel


In [8]:
# read data from "./data/db.sqlite3"
conn = sqlite3.connect("./data/db.sqlite3")
# create cursor
cur = conn.cursor()
# read data from strains_strain table
cur.execute("SELECT * FROM strains_strain")
# fetch all data
data = cur.fetchall()


In [6]:
data[0]


(1,
 'Green Crack',
 'green-crack',
 'Green Crack is a highly energetic and uplifting sativa-dominant strain known for its sharp focus and mental stimulation. The effects of Green Crack are invigorating, making it a popular choice for daytime use to boost creativity and productivity. This strain has a unique combination of sweet and tangy flavors with hints of citrus and tropical fruits. Green Crack is also valued for its ability to relieve stress and fatigue, providing a burst of energy without the typical crash associated with other strains.',
 'Sativa',
 '2024-03-14 13:40:17.463182',
 '2024-03-17 21:39:09.294111',
 'Green Crack is a legendary sativa strain known for its energizing and uplifting effects. Its bright green buds are covered in orange hairs and glistening trichomes, giving off a sweet and tangy aroma that instantly invigorates the senses. When smoked, Green Crack delivers a powerful burst of mental clarity and creativity, perfect for sparking inspiration and motivation. 

In [15]:
print(len(data[0]))


15


In [16]:
class Strain(BaseModel):
    sid: int
    name: str
    wikileaf_slug: str
    description: str
    strain_type: str
    stoner_description: str
    poem: str
    cons: list[str]
    effects: list[str]
    flavors: list[str]
    medical_use: list[str]
    pros: list[str]
    one_liner: str


### Create a validation class

In [23]:
cleaned_data: list[Strain] = []
for row in data:
    strain = Strain(
        sid=row[0],
        name=row[1],
        wikileaf_slug=row[2],
        description=row[3],
        strain_type=row[4],
        stoner_description=row[7],
        poem=row[8],
        cons=eval(row[9]),
        effects=eval(row[10]),
        flavors=eval(row[11]),
        medical_use=eval(row[12]),
        pros=eval(row[13]),
        one_liner=row[14],
    )
    cleaned_data.append(strain)


In [ ]:
cleaned_data


### Save Validated Data in json

In [27]:
# write cleaned data to "./data/strains.json"
import json

with open("./data/strains.json", "w") as f:
    json.dump(list(map(lambda data: dict(data), cleaned_data)), f, indent=4)
